# ReAct REPL Agent

Large-language model based "agent" that can can execute small tasks by using a Python REPL and pre-implemented methods. The "agent loop" is based on [ReAct](https://arxiv.org/abs/2210.03629).

The agent can look up methods using a `method_search()` method that uses semantic search on the indexed Python methods. Methods are indexed using OpenAI embedding of their signatures and descriptions.

To run this notebook make sure all secrets are in place to be able to access all APIs.

In [ ]:
# %load_ext autoreload
# %autoreload 2

In [ ]:
import sys
import logging
from pathlib import Path
from datetime import datetime, timedelta
import time

from openai import OpenAIError

In [ ]:
logger = logging.getLogger("root")
logger.setLevel(logging.INFO)
logger.addHandler(logging.StreamHandler(sys.stderr))

In [ ]:
from react_repl_agent.utils.output_format_logger import MdLogger
from react_repl_agent.llm_api import set_model, get_nb_tokens
from react_repl_agent.agent_steps.react_steps import (
    react_step_1,
    ReActFinished,
    get_react_step,
)
from react_repl_agent.methods import get_method_index
from react_repl_agent.apis.location import get_user_location
from react_repl_agent.utils.timedelta import format_timedelta

## Notebook Parameters

In [ ]:
llm_model = ""  # E.g. "gpt-4"
task_name = ""
task_description = ""

In [ ]:
print(f"{llm_model=}")
print(f"{task_name=}")
print(f"{task_description=!r}")

## Select OpenAI Model

In [ ]:
set_model(llm_model)

## Create method index

In [ ]:
method_search, context_dict = get_method_index()

## Define Task

## ReAct Loop

In [ ]:
start_time = time.time()
md_log = MdLogger()
# Get prompt to start with
md_log.display(f"# Task: {task_name}")
md_log.display(task_description)
md_log.display(f"Date: {datetime.now().strftime('%Y-%m-%d  %H:%M:%S %z')}")
md_log.display(f"Model: {llm_model}")

md_log.display("\n## Initial Prompt")
prompt_state, step_idx = react_step_1(
    task=task_description,
    method_search_fun=method_search,
    current_loc_method=get_user_location,
    context_dict=context_dict,
)
md_log.display_react(prompt_state)

# Run the ReAct THOUGHT/ACTION/OBSERVATION loop till the task is finished
max_steps = 20
for i in range(step_idx, max_steps+1):
    md_log.display(f"\n## STEP {i} - prompt size = {get_nb_tokens(prompt_state)}")
    try:
        thought, action, observation, prompt_state = get_react_step(
            prompt_state=prompt_state,
            step_idx=i,
            context_dict=context_dict,
            method_search_fun=method_search,
        )
    except ReActFinished as exc:
        md_log.display(f"### THOUGHT {i}:")
        md_log.display(exc.thought)
        md_log.display(f"### ACTION {i}:")
        md_log.display(f"```python\n{exc.action}\n```")
        prompt_state += f"THOUGHT {i}:\n{exc.thought}\nACTION {i}:\n```python\n{exc.action}\n```\n"""
        total_runtime = timedelta(seconds=time.time() - start_time)
        md_log.display(f"`stop()` called! Agent finished in {i} steps and {format_timedelta(total_runtime)}.")
        md_log.display(str(exc))
        break
    except OpenAIError as exc:
        md_log.display(f"## ERROR!")
        md_log.display(f"{type(exc).__name__}: {exc!s}")
        prompt_state += f"\n...\nERROR {type(exc).__name__}: {exc!s}."""
        logging.exception(exc)
        break
    md_log.display(f"### THOUGHT {i}:")
    md_log.display(thought)
    md_log.display(f"### ACTION {i}:")
    md_log.display(f"```python\n{action}\n```")
    md_log.display(f"### OBSERVATION {i}:")
    md_log.display(f"{observation}")

## Export full prompt history

In [ ]:
# Export
export_dir = Path("../react_logs").resolve()
now = datetime.now()
export_name = f"{task_name}__{llm_model}__{now.strftime('%Y_%m_%d__%H-%M-%S')}.txt"
export_path_plain = export_dir / llm_model / now.strftime('%Y') / now.strftime('%m') / now.strftime('%d') / export_name
export_path_plain.parent.mkdir(parents=True, exist_ok=True)
# Export plaintext
with export_path_plain.open("w") as f_out:
    f_out.write(prompt_state.strip())
# Export Markdown run log
export_path_md = export_path_plain.with_suffix(".md")
with export_path_md.open("w") as f_out:
    f_out.write(md_log.log.strip())

print(f"Exported run logs to '{export_path_plain}'")